# Yield to Maturity

### [Definition](https://www.investopedia.com/terms/y/yieldtomaturity.asp)

- Yield to maturity (YTM) is the total return anticipated on a bond if the bond is held until it matures.
    - YTM is a *long-term* rate of return
        - Expressed annually
- In other words, the **YTM is the IRR for the bond**

- **Note:** YTM is also called *book yield* or *redemption yield*

____

## How is this different from Current Yield?

- The current yield of a bond is the **annual income of the bond as a fraction of the market price**
    - **Note**: Current Yield uses market price instead of face value in the calculation
    
#### Why do we care about current yield?

- It tells us what our return will be if we buy the bond today and sell it in a year

#### Is this not the same as the coupon?

- No
    - **Why?**
        - Because we're using the market price instead of the face value
        
#### Examples

- If we buy a 6% coupon bond at a discount of \$900, then our current yield will be:

$$
\left ( \text{Current Yield} \right ) = \frac{0.06\cdot 1000}{900} = 0.0667
$$

- Now, assume we bought the same bond for \$1,100 instead of \$900

$$
\left ( \text{Current Yield} \right ) = \frac{0.06\cdot 1000}{1100} = 0.0545
$$

### Therefore, compared to the Current Yield, 1) YTM considers more than just the next year and 2) unlike CY, YTM accounts for the time value of money (by using a discount rate in the calculation)

- For these reasons, YTM is considered to be a more thorough measure of the return for a bond

____

## Calculating YTM

- YTM assumes that every coupon received is invested at a constant interest rate until the maturity date
    - Therefore, we take the present value of each coupon to calculate the market price of the bond
    
### Example

- Let's say we have a bond whose face value is \$100
    - The bond is currently priced at a discount of \$95.92
    - The bond matures in 30 months
    - The bond pays a semi-annual coupon of 5%
    
- This means that the current yield of the bond is equal to:

$$
\left ( \text{Current Yield} \right ) = \frac{0.05\cdot100}{95.52} = 0.052345
$$

- To calculate YTM, let's build out the cashflows
    - We know we'll receive a coupon of $0.05\cdot100/2 = 2.50$ every six months until the maturity of the bond
        - We'll also receive the \$100 face value at maturity

In [26]:
import pandas as pd
import numpy as np

In [23]:
df_cashflows = pd.DataFrame(columns = ['Coupons', 'Face Value'], index = range(1,31))

for month in range(1,31):
    if month % 6 == 0:
        df_cashflows.loc[month, 'Coupons'] = 2.50
    else:
        df_cashflows.loc[month, 'Coupons'] = 0
    if month == 30:
        df_cashflows.loc[month, 'Face Value'] = 100
    else:
        df_cashflows.loc[month, 'Face Value'] = 0

df_cashflows['Total'] = df_cashflows.sum(axis=1)

In [24]:
df_cashflows

,Coupons,Face Value,Total
1,0,0,0.0
2,0,0,0.0
3,0,0,0.0
4,0,0,0.0
5,0,0,0.0
6,2.5,0,2.5
7,0,0,0.0
8,0,0,0.0
9,0,0,0.0
10,0,0,0.0


### So now, we need to solve for the discount rate so that the present value of the Total column is equal to the market price of \$95.92

### i.e. we need to solve for YTM in the following equation:

$$
95.92 = \frac{2.5}{(1+YTM)^{6}}+\frac{2.5}{(1+YTM)^{12}}+\frac{2.5}{(1+YTM)^{18}}+\frac{2.5}{(1+YTM)^{24}}+\frac{102.5}{(1+YTM)^{30}}
$$

- There are many solver algorithms we can use to solve for YTM, but for now we'll use a simple loop
    - We know that **when a bond is priced at a par, the YTM is equal to the coupon**
        - When a bond is **priced at a discount, the YTM is GREATER than the coupon**
        
- So what we'll do is: we'll start at a YTM of 5%, and we'll keep adjusting it and recalculating the present value until we approximate the YTM

In [45]:
stop = False

YTM = 0.05
min_difference = 100

totals_array = df_cashflows['Total'].values

while not stop:
    monthly_YTM = YTM/12.0
    discount_array = 1/((1 + monthly_YTM)**np.arange(1,31))
    PV_array = totals_array*discount_array
    PV = np.sum(PV_array)
    difference = abs(PV - 95.92)
    if difference < min_difference:
        min_difference = difference
        YTM += 0.00001
    if difference > min_difference:
        stop = True
YTM, PV, difference

(0.06709000000000205, 95.91725456696676, 0.0027454330332403742)

### As we can see, our loop solved for an approximate YTM of 0.06709 which gives an approximated PV of 95.917 (close enough)

### Let's repeat the example, but this time with a market price of 105 for the bond

- This means we'll start with a YTM of 0.05, but we'll **subtract** from it each loop (instead of adding)

In [46]:
stop = False

YTM = 0.05
min_difference = 100

totals_array = df_cashflows['Total'].values

while not stop:
    monthly_YTM = YTM/12.0
    discount_array = 1/((1 + monthly_YTM)**np.arange(1,31))
    PV_array = totals_array*discount_array
    PV = np.sum(PV_array)
    difference = abs(PV - 105)
    if difference < min_difference:
        min_difference = difference
        YTM -= 0.00001
    if difference > min_difference:
        stop = True
YTM, PV, difference

(0.02892999999999436, 105.00323684475313, 0.003236844753132573)

### In this example, the YTM is 0.02892

____

## Uses of YTM

- YTM is pretty good for checking whether a bond is a good investment or not
    - If an investor doesn't want to invest in anything that has a return below 5% (i.e. their required return is 5%) and the YTM is 3%, then they'll know to not invest
    
- Also, since the YTM is an annualized rate, it's a good yardstick for measuring bonds of different maturities

____

## Variations of YTM

1. **Yield to Call (YTC)**
    - assumes that the bond will be called
        - i.e. the bond will be bought back by the issuer before the maturity date
    - for this calculation, we shift the repayment of the face value back to our estimated call date, and drop all coupon payments after that point
        - Then we calculate YTM as we did above
        
2. **Yield to Put (YTP)**
    - 